In [ ]:
import pickle 
import os
from datetime import datetime,timezone,timedelta
import pickle

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tqdm import tqdm
from scipy import signal

import data_filter as dafi
import data_process as dapr
import lib_plot

In [ ]:
azimut_mask = [270,330]
elevation_mask = [3,30]
min_height = 2
max_height = 4
t_range = 30
main_path_2 = os.path.abspath('../data/20220904/2/')
main_path_3 = os.path.abspath('../data/20220904/3/')

main_path = main_path_2
month = 9
day = 4

In [ ]:
frequency = dapr.generate_frequency(min_height=min_height,max_height=max_height)
signal_ts = {}

for dt in tqdm(range(0,t_range-1)):
    starttime = datetime(year=2022,month=month,day=day,hour=8,\
        minute=0+dt,second=0,tzinfo=timezone.utc)
    endtime = datetime(year=2022,month=month,day=day,hour=14,\
        minute=0+dt,second=0,tzinfo=timezone.utc)
    deltatime = timedelta(minutes=t_range)

    data_dict = dafi.clean_data(main_path,azimut_mask=azimut_mask,\
        elevation_mask=elevation_mask,sn1_trigger=True)

    split_data_dict = dafi.split_data(data_dict,starttime,endtime,deltatime)
    result_dict = dapr.data_prepare(split_data_dict,frequency=frequency)
    result_dict_1031 = dapr.extract_height_as_max_peak(result_dict,frequency=frequency)

    for satellite_code in result_dict_1031:
        for t,p in zip(result_dict_1031[satellite_code]['time'],\
            result_dict_1031[satellite_code]['power']):
            if t in signal_ts:
                signal_ts[t] = np.multiply(p,signal_ts[t])
            else:
                signal_ts[t] = p
height_ts = {}
for t in signal_ts:
    p = signal_ts[t]
    index = p==max(p)
    h = frequency[index]
    height_ts[t] = h[0]